In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dreams = pd.read_csv("dreams.csv", index_col=0)

In [ ]:
dreams.date = pd.to_datetime(dreams.date)

In [ ]:
dreams["month"] = dreams.date.astype("datetime64[M]")
dreams["quarter"] = dreams.date.dt.to_period("Q")
dreams["day"] = dreams.date.dt.weekday
dreams["characters"] = dreams.content.str.len()

In [ ]:
content_filter = (dreams["content"] != "no memory") & (dreams["content"] != "weak memory")

In [ ]:
plt.plot(dreams.groupby("month").count()["date"])
plt.plot(dreams[content_filter].groupby("month").count()["date"])

In [ ]:
dreams1 = dreams[dreams.date < "2021-04-01"]
dreams2 = dreams[dreams.date >= "2021-04-01"]

In [ ]:
dreams.shape[0]

In [ ]:
dreams = dreams[content_filter]

In [ ]:
from ast import literal_eval
dreams.people = dreams.people.apply(literal_eval)
dreams.female = dreams.female.apply(literal_eval)
dreams.category = dreams.category.apply(literal_eval)
dreams.locations = dreams.locations.apply(literal_eval)
dreams.entities = dreams.entities.apply(literal_eval)

In [ ]:
gender = dreams.explode("female").groupby("female").count()["date"]
#gender["nan"] = dreams.explode("female")["female"].isna().sum()
gender = gender.rename({False: "male", True: "female"}, axis=1)
gender

In [ ]:
fig, ax = plt.subplots()
ax.pie(gender, labels=gender.index, autopct='%1.1f%%')
ax.set_title("Gender");

In [ ]:

categories = dreams.explode("category").groupby("category").count()["date"].sort_values(ascending=False)
categories = categories[categories.values > 11]
fig, ax = plt.subplots()
ax.pie(categories, labels=categories.index, autopct='%1.1f%%')
ax.set_title("Categories");

In [ ]:

categories = dreams.explode("category")
categories = categories[(categories["category"] != "no memory") & (categories["category"] != "weak memory")].groupby("category").count()["date"].sort_values(ascending=False)
categories = categories[categories.values > 11]
fig, ax = plt.subplots()
ax.pie(categories, labels=categories.index, autopct='%1.1f%%')
ax.set_title("Categories");

In [ ]:
people = dreams.explode("people").groupby("people").count()["date"]
people

In [ ]:
people.index.unique()

In [ ]:
parties = dreams[dreams.category.apply(lambda x: "partying" in x)].groupby("date").count()["content"]
idx = pd.date_range(dreams.date.min(), dreams.date.max())
parties = parties.reindex(idx, fill_value=0)
parties.plot()

In [ ]:
parties = dreams[dreams.category.apply(lambda x: "partying" in x)].groupby("month").count()["content"]
parties.plot()

In [ ]:
parties = dreams1[dreams1.category.apply(lambda x: "sex" in x)]
parties = parties.groupby("quarter").count()["content"] / dreams1.groupby("quarter").count()["content"]
parties.plot()

In [ ]:
sex = dreams[dreams.category.apply(lambda x: "partying" in x)]
sex.groupby("day").count()["date"]

In [ ]:
dreams.groupby("day").mean()["characters"]

In [ ]:
dreams

In [ ]:
categories = dreams.explode("female")
categories = categories.explode("category")
categories.female = categories.female.astype(bool)
undefined = dreams[dreams.female.apply(lambda x: len(x)) == 0].explode("category")
gender = categories[categories.female.notna()]
female = gender[gender.female]
male = gender[~gender.female]
index = categories.groupby("category").count()["date"].sort_values(ascending=False).index
female = female.groupby("category").count()["date"]
female = female.reindex(index, fill_value=0)
male = male.groupby("category").count()["date"]
male = male.reindex(index, fill_value=0)
male += female
undefined = undefined.groupby("category").count()["date"]
undefined = undefined.reindex(index, fill_value=0)
undefined += male
plt.bar(index, undefined);
plt.bar(index, male);
plt.bar(index, female);
plt.xticks(rotation=90);

In [ ]:
dreams.columns

In [ ]:
dreams.explode("locations").groupby("locations").count()["date"].sort_values(ascending=False).head(20)

In [ ]:
dreams.explode("entities").groupby("entities").count()["date"].sort_values(ascending=False).head(20)

In [ ]:
dreams.explode("people")["people"].unique()[60:]

In [ ]:
dreams.content.sample(1).iloc[0]